In [1]:
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
import numpy as np
import sys 
import pandas as pd
from datetime import datetime

In [5]:
!head data/train.txt

1,822109,5,2005-05-13
1,885013,4,2005-10-19
1,30878,4,2005-12-26
1,893988,3,2005-11-17
1,124105,4,2004-08-05
1,1248029,3,2004-04-22
1,1842128,4,2004-05-09
1,2238063,3,2005-05-11
1,1503895,4,2005-05-19
1,2207774,5,2005-06-06


In [16]:
def train_generator ():
    train  = pd.read_csv ('data/train.txt',names=['mid', 'uid', 'rate', 'date'])
    count = len (train)
    min_date = datetime.strptime('1999-11-10', '%Y-%m-%d')
    train['date_day'] = train.apply(lambda x: (datetime.strptime(x['date'], '%Y-%m-%d') - min_date).days , axis=1)
    print "days updated"
    train = train.sort_values (by=['uid', 'date_day'])
    train = train.reset_index (drop=True)
    print "sorted"
    mids = train.mid
    mids.loc[-1] = None
    mids.index = mids.index + 1
    mids.sort_index (inplace=True)
    mids = mids.drop(mids.index[len(mids)-1])
    print "mids constructed"
    fisrt_inds = []
    for uid in set (train.uid): 
        fisrt_inds.append (train[train.uid == uid].index[0])
    mids[fisrt_inds] = None
    print "mids really constructed"
    train['prev_movie'] = mids
    keys = ['mid', 'uid', 'time', 'prev']
    cur_num = 0
    for uid in set (train.uid):
        group = train.groupby ('uid').get_group (uid)
        val = 1. / len (group)
        other_movies = dict (zip (group.mid, [val] * len (group)))
        for ind in range (len (group)):
            other_movies.update (dict (zip (keys, group.iloc[ind][['mid', 'uid', 'date_day', 'prev_movie']])))
            if other_movies['prev_movie'] is None:
                other_movies.pop ('prev_movie')
            cur_num += 1
            sys.stderr.write ("\r{}/{}".format (cur_num, count))
            y.append (group.iloc[ind].rank)
            yield other_movies

In [17]:
v = DictVectorizer()

In [ ]:
X = v.fit_transform (train_generator ())

days updated
sorted
mids constructed


In [9]:

train  = pd.read_csv ('data/train.txt',names=['mid', 'uid', 'rate', 'date'])

test = pd.read_csv ('data/test.txt',names=['mid', 'uid', 'date'])

In [12]:
len (set (test.mid))

17770

In [11]:
len (set (test.uid))

472373